In [7]:
import os
import pandas as pd
from transformers import CLIPProcessor, CLIPModel, AutoProcessor
import torch
from PIL import Image
import numpy as np

# from collections import defaultdict
# from sklearn.metrics import mean_squared_error
# from sklearn.metrics.pairwise import cosine_similarity
# import pandas as pd
# import matplotlib.pyplot as plt
# import matplotlib.patches as P
# from tqdm import tqdm
# import csv
# import torch.nn.functional as F

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [8]:
# Load CLIP model and processor
model_name = "openai/clip-vit-large-patch14"
processor = AutoProcessor.from_pretrained(model_name)
model = CLIPModel.from_pretrained(model_name).to(device)
model.eval()

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.


CLIPModel(
  (text_model): CLIPTextTransformer(
    (embeddings): CLIPTextEmbeddings(
      (token_embedding): Embedding(49408, 768)
      (position_embedding): Embedding(77, 768)
    )
    (encoder): CLIPEncoder(
      (layers): ModuleList(
        (0-11): 12 x CLIPEncoderLayer(
          (self_attn): CLIPAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (layer_norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): CLIPMLP(
            (activation_fn): QuickGELUActivation()
            (fc1): Linear(in_features=768, out_features=3072, bias=True)
            (fc2): Linear(in_features=3072, out_features=768, bias=True)
          )
          (layer_norm2): LayerNorm((768,), eps=1e-05,

# Embed Dataset

In [35]:
def get_image_embeddings(images):
    inputs = processor(images=images, return_tensors="pt", padding=True)
    inputs = inputs.to(device)
    with torch.no_grad():
        image_features = model.get_image_features(pixel_values=inputs['pixel_values']) # Get the class token embedding 
    return image_features  

In [36]:
def compute_image_embeddings(dataset_name = 'CLEVR', batch_size=100):
    """Computes image embeddings, centers, and normalizes them."""
    metadata = pd.read_csv(f'Data/{dataset_name}/metadata.csv')
    image_paths = metadata['image_path'].tolist()

    # Load images
    print("Loading images...")
    images = []
    for image_filename in tqdm(image_paths):
        #image_path = os.path.join(image_dir, image_filename)
        image = Image.open(image_filename).convert("RGB")
        image_tensor = torch.tensor(np.array(image).transpose((2, 0, 1)))  # Convert image to (C, H, W)
        images.append(image_tensor)
    
    print(f"Loaded {len(images)} images.")

    # Embed images in batches
    print("Computing embeddings in batches...")
    embeddings = []
    n_batches = (len(images) + batch_size - 1) // batch_size  # Calculate number of batches
    for i in tqdm(range(n_batches)):
        batch_images = images[i * batch_size:(i + 1) * batch_size]
        batch_embeddings = get_image_embeddings(batch_images)  # Replace with your actual embedding function
        embeddings.append(batch_embeddings)
    
    embeddings = torch.cat(embeddings, dim=0).cpu()  # Concatenate all batch embeddings
    print(f"Extracted embeddings of shape: {embeddings.shape}")

    # Center embeddings
    print("Centering embeddings...")
    mean_embedding = embeddings.mean(dim=0)
    centered_embeddings = embeddings - mean_embedding
    
    # Normalize embeddings
    norm_embeddings = centered_embeddings / centered_embeddings.norm(dim=0, keepdim=True)

    # Save results
    output_file = f'Experiments/Embeddings/{dataset_name}_embeddings.pt'
    torch.save(norm_embeddings, output_file)
    print(f"Embeddings saved to {output_file} :)")

compute_image_embeddings()

Loading images...


100%|██████████| 1001/1001 [00:13<00:00, 72.54it/s]


Loaded 1001 images.
Computing embeddings in batches...


  0%|          | 0/11 [00:00<?, ?it/s]/opt/conda/envs/rapids/lib/python3.10/site-packages/torch/nn/modules/conv.py:459: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv2d(input, weight, bias, self.stride,
100%|██████████| 11/11 [00:12<00:00,  1.14s/it]


Extracted embeddings of shape: torch.Size([1001, 768])
Centering embeddings...
Embeddings saved to Experiments/Embeddings/CLEVR_embeddings.pt :)


# Compute Concepts

In [38]:
def compute_concepts(dataset_name = 'CLEVR'):
    # Load the embeddings from the .pt file
    embeddings = torch.load(input_embeddings_file)

    # Load the color and shape data from the CSV file
    metadata_df = pd.read_csv(input_metadata_file)
    colors = metadata_df['color'].tolist()
    shapes = metadata_df['shape'].tolist()

    # Create a dictionary to store the embeddings based on the chosen grouping
    concept_embeddings = {}

    # Iterate through the data and group embeddings based on the chosen criteria
    for idx, embedding in enumerate(embeddings):
        color = colors[idx]
        shape = shapes[idx]

        # Grouping logic
        if group_by == 'color':
            key = color
        elif group_by == 'shape':
            key = shape
        elif group_by == 'both':
            key = f'{color}_{shape}'
        else:
            raise ValueError("Invalid value for 'group_by'. Choose from 'color', 'shape', or 'both'.")

        # Add the embedding to the dictionary under the chosen key
        if key not in concept_embeddings:
            concept_embeddings[key] = []
        concept_embeddings[key].append(embedding)

    # Calculate the average embedding for each group
    avg_concept_embeddings = {}
    for key, group_embeddings in concept_embeddings.items():
        group_embeddings = torch.stack(group_embeddings)  # Convert to tensor
        avg_embedding = group_embeddings.mean(dim=0)  # Calculate the average embedding
        avg_embedding = avg_embedding / avg_embedding.norm()  # Normalize the average embedding
        avg_concept_embeddings[key] = avg_embedding

    # Save the average embeddings to a .pt file
    output_file = os.path.join(output_dir, f'concepts_{group_by}.pt')  # Changed to .pt
    torch.save(avg_concept_embeddings, output_file)

    print(f"Concept embeddings grouped by {group_by} saved at {output_file} :)")

# Example usage:
#compute_concepts()

torch.Size([1001, 768])
